# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846858043013                   -0.70    4.0
  2   -7.852325800244       -2.26       -1.53    1.0
  3   -7.852646090320       -3.49       -2.52    3.0
  4   -7.852646676392       -6.23       -3.34    2.5
  5   -7.852646685781       -8.03       -4.77    1.0
  6   -7.852646686725       -9.02       -5.16    4.2
  7   -7.852646686729      -11.36       -5.84    2.0
  8   -7.852646686730      -12.39       -7.25    2.0
  9   -7.852646686730      -14.75       -7.69    2.2
 10   -7.852646686730   +  -14.75       -8.83    2.5
 11   -7.852646686730      -14.75       -9.53    1.8
 12   -7.852646686730   +    -Inf      -10.24    1.8
 13   -7.852646686730      -14.75      -10.88    2.2
 14   -7.852646686730   +  -14.75      -11.32    2.0
 15   -7.852646686730      -14.75      -11.91    1.8
 16   -7.852646686730   +  -14.75      -12.75    2.8


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846897092033                   -0.70           4.0
  2   -7.852531712227       -2.25       -1.63   0.80    2.0
  3   -7.852637654019       -3.97       -2.71   0.80    1.0
  4   -7.852646466853       -5.05       -3.30   0.80    2.2
  5   -7.852646682302       -6.67       -4.15   0.80    1.5
  6   -7.852646686384       -8.39       -4.80   0.80    1.2
  7   -7.852646686723       -9.47       -5.84   0.80    1.8
  8   -7.852646686730      -11.19       -6.77   0.80    2.2
  9   -7.852646686730      -12.72       -7.62   0.80    2.2
 10   -7.852646686730   +    -Inf       -8.64   0.80    1.5
 11   -7.852646686730      -15.05       -8.34   0.80    3.0
 12   -7.852646686730   +  -15.05       -9.19   0.80    1.0
 13   -7.852646686730      -14.75      -10.34   0.80    1.0
 14   -7.852646686730   +  -15.05      -10.45   0.80    3.0
 15   -7.852646686730      -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.417891e+01     3.565430e+00
 * time: 0.4006030559539795
     1     1.487755e+00     1.799995e+00
 * time: 0.5968949794769287
     2    -1.671022e+00     2.061087e+00
 * time: 0.6202530860900879
     3    -3.999604e+00     1.743739e+00
 * time: 0.6540060043334961
     4    -5.576642e+00     1.430883e+00
 * time: 0.6874790191650391
     5    -7.018948e+00     6.941714e-01
 * time: 0.7212250232696533
     6    -7.394014e+00     6.262683e-01
 * time: 0.7444601058959961
     7    -7.602265e+00     1.506265e-01
 * time: 0.7681078910827637
     8    -7.677591e+00     1.641934e-01
 * time: 0.7913539409637451
     9    -7.742363e+00     1.573845e-01
 * time: 0.8146641254425049
    10    -7.785356e+00     1.043321e-01
 * time: 0.8379449844360352
    11    -7.819385e+00     7.046468e-02
 * time: 0.8614609241485596
    12    -7.836699e+00     4.126218e-02
 * time: 0.884645938873291
    13    -7.843618e+00     3.722691e-02
 * time: 0.90810799598

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846825016172                   -0.70    4.0
  2   -7.852318158115       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686690                   -2.54
  2   -7.852646686730      -10.40       -5.84
  3   -7.852646686730   +  -14.75      -12.57


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 9.254793092100821e-14
|ρ_newton - ρ_scfv| = 2.3576073220825363e-11
|ρ_newton - ρ_dm|   = 3.2364494451510676e-10
